In [1]:
import os 
import librosa
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Dropout,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,SGD
from transformers import  WhisperProcessor
from tensorflow.keras.regularizers import l2
import pandas as pd
import numpy as np
from sklearn import metrics 
from sklearn.model_selection import train_test_split
import mlflow
import warnings
warnings.filterwarnings('ignore')

2024-07-23 10:47:07.970640: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 10:47:07.973902: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 10:47:07.985947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 10:47:08.008586: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 10:47:08.008627: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 10:47:08.024397: I tensorflow/core/platform/cpu_feature_guard.cc:

In [3]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('Using MFCC from Librosa')


2024/07/23 10:48:17 INFO mlflow.tracking.fluent: Experiment with name 'Using MFCC from Librosa' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/670346074458369401', creation_time=1721746097304, experiment_id='670346074458369401', last_update_time=1721746097304, lifecycle_stage='active', name='Using MFCC from Librosa', tags={}>

In [133]:
mlflow.tensorflow.autolog()

2024/07/19 07:49:38 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


In [5]:
import os
import librosa

folder_path = '/home/dylan/HealthDetectionThroughVoice/patient-vocal-dataset'
label = []
k =0
audio_data = []

for folder in os.listdir(folder_path):
    audio_folder_path = os.path.join(folder_path,folder)
    for filename in os.listdir(audio_folder_path):
        if filename.endswith('.wav') :
            audio_file_path = os.path.join(audio_folder_path,filename)
            audio,_ = librosa.load(audio_file_path,sr = 16000)
            audio_data.append(audio)
            label.append(k)
    k += 1


In [6]:
os.listdir(folder_path)

['Normal', 'Vox senilis', 'Laryngozele']

In [7]:
def feature_extraction(audio,sr):
    #features = feature_extractor(audio,sampling_rate=sr,return_tensors='np').input_features[0]
    features = librosa.feature.mfcc(y = audio, sr = sr,n_mfcc= 50)
    transpose_feature = np.transpose(features,(1,0))
    average_scaled_features = np.mean(transpose_feature, axis = 0)
    return average_scaled_features

In [8]:
features = []
for audio in audio_data :
    features.append(feature_extraction(audio,_))

In [9]:
len(features)

1036

In [10]:
data = pd.DataFrame(data = features)

In [11]:
data['labels'] = label

In [12]:
data['labels'].unique()

array([0, 1, 2])

In [13]:
x = data.drop('labels', axis =1)
y = np.array(data['labels'].tolist())
y = np.array(pd.get_dummies(y))
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [14]:
x

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-160.685165,45.490135,-4.821774,35.931957,-10.787026,15.464179,-8.226496,6.952154,-5.721563,1.627728,...,-21.462183,9.511221,12.668848,-13.635872,-23.943476,5.275884,11.726404,-15.634768,-29.240330,-1.091166
1,-312.905090,106.385986,30.501062,31.441315,-9.121834,-9.406259,-3.091830,4.112461,-11.861338,0.447344,...,2.208146,1.817251,-0.181122,0.120193,-0.497871,2.098290,-1.448841,-1.255415,-1.664657,-2.020061
2,-198.047699,144.320862,17.916195,21.477489,10.471588,13.473979,2.826607,7.292999,1.832671,5.262392,...,-1.524959,0.355890,-3.344779,-0.008432,-0.059242,5.186929,5.414943,8.438732,6.083746,7.030056
3,-277.448975,88.948769,29.334581,24.219416,12.109037,5.466475,-8.748367,14.119501,-14.876341,-8.853692,...,-1.082283,3.768034,0.810410,4.173285,0.778366,4.612110,-0.353598,3.696862,2.749008,5.950413
4,-298.540070,158.716324,29.298662,-3.409457,-25.199650,-1.150077,-11.963250,-0.317667,4.107648,-6.450688,...,40.711491,51.524853,30.746332,15.714661,-3.517975,-1.933229,2.197755,5.977923,-0.656958,-6.218029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,-249.958893,122.502533,73.687592,-14.402223,11.862350,-20.625967,-11.463581,25.574898,3.157075,-3.082764,...,-5.997714,-6.161963,1.260105,-2.781147,7.297731,12.011082,15.292834,25.896961,11.474255,17.393736
1032,-211.993591,55.510059,-40.787830,93.921974,19.720709,-42.127529,-7.178773,-0.600349,-25.551603,-15.691953,...,-0.576841,-6.795305,-8.308597,9.469365,7.675497,-3.163553,-8.838223,-3.405991,3.822929,5.032875
1033,-137.966064,135.945160,-12.244995,-47.050213,2.576136,-12.112126,-4.900474,11.373034,3.213206,-4.643087,...,-8.791137,-1.471322,-7.373990,-0.827134,-11.440187,-0.615440,-5.766508,-5.503603,-2.432851,-3.399486
1034,-168.714355,82.739861,24.001690,17.513767,6.689113,4.423263,7.146572,7.735805,5.888155,9.388286,...,-4.453273,0.055080,-2.465599,-1.956569,-8.699000,-9.479540,-8.938299,5.262674,21.440670,39.075733


In [27]:
mlflow.start_run()

Exception: Run with UUID 5516b190dcd14d9bb3b3f8191ad6558e is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [28]:
model = Sequential()

## adding the input layer 
model.add(Input(shape = (50,), batch_size = 16))
## adding the first Dense Layer 
model.add(Dense(32,activation = 'relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.075)))
## adding a dropout layer 
#model.add(Dropout(0.1))
## layer 2
model.add(Dense(16, activation = 'relu', kernel_initializer = 'he_uniform', kernel_regularizer=l2(0.075)))
#model.add(Dropout(0.1))
## layer 3
#model.add(Dense(16, activation = 'relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.05)))
#model.add(Dropout(0.1))

model.add(Dense(8,activation = 'relu', kernel_initializer='he_uniform', kernel_regularizer= l2(0.075)))
#model.add(Dropout(0.5))
#model.add(Dense(64,activation = 'relu', kernel_initializer='he_uniform'))

## layer 4

model.add(Dense(8,activation= 'relu', kernel_initializer = 'he_uniform', kernel_regularizer=l2(0.075)))
#model.add(Dropout(0.1))

model.add(Dense(3, activation = 'softmax'))

In [29]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (16, 32)               │         1,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (16, 16)               │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (16, 8)                │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (16, 8)                │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (16, 3)                │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,395 (9.36 KB)

 Trainable params: 2,395 (9.36 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
params={'learning_rate' : 0.005, 'epsilon' : 1e-07}
optimizer = Adam(**params)
model.compile(optimizer= optimizer, loss = 'categorical_crossentropy',metrics=['accuracy'])

In [31]:
history = model.fit(X_train,y_train,epochs = 200, validation_data=(X_test,y_test))

for epoch in range(len(history.history['loss'])):
    training_loss = history.history['loss'][epoch]
    validation_loss = history.history['val_loss'][epoch]
    validation_accuracy = history.history['val_accuracy'][epoch]
    training_accuracy = history.history['accuracy'][epoch]

    mlflow.log_metric('training__loss', training_loss, step = epoch)
    mlflow.log_metric('training_accuracy', training_accuracy, step = epoch)
    mlflow.log_metric('validation_accuracy',validation_accuracy,step=epoch)
    mlflow.log_metric('validation_loss',validation_loss, step=epoch)

mlflow.tensorflow.log_model(model,'model')
mlflow.log_artifact('/home/dylan/HealthDetectionThroughVoice/HealthDetection.ipynb','source_code')

Epoch 1/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4848 - loss: 18.4487 - val_accuracy: 0.6058 - val_loss: 8.6563
Epoch 2/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5816 - loss: 7.7399 - val_accuracy: 0.6250 - val_loss: 5.9393
Epoch 3/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6299 - loss: 5.5727 - val_accuracy: 0.6587 - val_loss: 4.6533
Epoch 4/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6749 - loss: 4.3681 - val_accuracy: 0.6971 - val_loss: 3.8225
Epoch 5/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7227 - loss: 3.5967 - val_accuracy: 0.6923 - val_loss: 3.2175
Epoch 6/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7266 - loss: 3.0986 - val_accuracy: 0.7212 - val_loss: 2.8539
Epoch 7/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7509 - loss: 2.6788 - val_accuracy: 0.7212 - val_loss: 2.5470
Epoch 8/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7437 - loss: 2.4442 - val_accuracy: 0.7308 -

2024/07/23 10:51:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


In [32]:
mlflow.end_run()

In [33]:
import pickle 

In [34]:
pickle.dump(model,open('nn_model.pkl','wb'))